In [1]:
import csv
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error
# Importamos utilidades y modelos de sklearn
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
import category_encoders as ce

In [2]:
dtypes = {'id': 'int32', 
'titulo': 'object', 
'descripcion': 'object', 
'tipodepropiedad': 'category', 
'direccion': 'object', 
'ciudad': 'object', 
'provincia': 'category', 
'antiguedad': 'float', 
'habitaciones': 'float', 
'garages': 'float', 
'banos': 'float', 
'metroscubiertos': 'float', 
'metrostotales': 'float', 
'idzona': 'object', 
'lat': 'float64', 
'lng': 'float64', 
'gimnasio': 'float', 
'usosmultiples': 'float', 
'piscina': 'float', 
'escuelascercanas': 'float', 
'centroscomercialescercanos': 'float', 
'precio': 'float', }

data = pd.read_csv("train.csv", dtype = dtypes)
data.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos,precio
0,254099,depto. tipo a-402,"depto. interior de 80.15m2, consta de sala com...",Apartamento,Avenida Division del Norte 2005,Benito Juárez,Distrito Federal,NaN,2.0,1.0,...,23533.0,NaN,NaN,2015-08-23 00:00:00,0.0,0.0,0.0,0.0,0.0,2273000.0
1,53461,condominio horizontal en venta,"<p>entre sonora y guerrero, atr&aacute;s del h...",Casa en condominio,AV. MEXICO,La Magdalena Contreras,Distrito Federal,10.0,3.0,2.0,...,24514.0,19.310205,-99.227655,2013-06-28 00:00:00,0.0,0.0,0.0,1.0,1.0,3600000.0
2,247984,casa en venta urbi 3 recamaras tonala,descripcion \nla mejor ubicacion residencial e...,Casa,Urbi Tonala,Tonalá,Jalisco,5.0,3.0,2.0,...,48551.0,NaN,NaN,2015-10-17 00:00:00,0.0,0.0,0.0,0.0,0.0,1200000.0
3,209067,casa sola en toluca zinacantepec con credito i...,casa en privada con caseta de vigilancia casas...,Casa,IGNACIO MANUEL ALTAMIRANO 128,Zinacantepec,Edo. de México,1.0,2.0,1.0,...,53666.0,19.301890,-99.688015,2012-03-09 00:00:00,0.0,0.0,0.0,1.0,1.0,650000.0
4,185997,paseos del sol,bonito departamento en excelentes condiciones ...,Apartamento,PASEOS DEL SOL,Zapopan,Jalisco,10.0,2.0,1.0,...,47835.0,NaN,NaN,2016-06-07 00:00:00,0.0,0.0,0.0,0.0,0.0,1150000.0


# Pre-procesamiento de data

In [3]:
data["ciudad"].nunique()

875

In [4]:
data.isnull().sum()

id                                 0
titulo                          5387
descripcion                     1619
tipodepropiedad                   46
direccion                      53072
ciudad                           372
provincia                        155
antiguedad                     43555
habitaciones                   22471
garages                        37765
banos                          26221
metroscubiertos                17400
metrostotales                  51467
idzona                         28621
lat                           123488
lng                           123488
fecha                              0
gimnasio                           0
usosmultiples                      0
piscina                            0
escuelascercanas                   0
centroscomercialescercanos         0
precio                             0
dtype: int64

In [5]:
data.isnull().sum().sum()

535127

In [6]:
data.size

5520000

In [7]:
(data.isnull().sum().sum()/(data.size))*100

9.694329710144928

Los Nulls representan el 10% de los datos.

# CON XGBOOST NO HACE FALTA PREOCUPARSE POR LOS NULLs

# Levanto el csv de test para calcularle los features en paralelo

In [8]:
test = pd.read_csv("test.csv", dtype = dtypes)
test.head()

,id,titulo,descripcion,tipodepropiedad,direccion,ciudad,provincia,antiguedad,habitaciones,garages,...,metrostotales,idzona,lat,lng,fecha,gimnasio,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,"casa en venta en miguel hidalgo, distrito federal",<p>excelente casa estilo moderno.</p>,Casa,Bosque de Cedros,Miguel Hidalgo,Distrito Federal,29.0,3.0,NaN,...,NaN,NaN,19.408668,-99.246767,2013-07-20 00:00:00,0.0,0.0,0.0,0.0,0.0
1,51775,departamentos en venta en montebello,<p>departamento una recamara:\n</p><p>departam...,Apartamento,NaN,Mérida,Yucatán,NaN,1.0,1.0,...,67.0,113851.0,21.032480,-89.592424,2015-10-24 00:00:00,0.0,0.0,0.0,0.0,0.0
2,115253,departamento nuevo delegación coyoacán de 87 m...,"departamento nuevo de 87.06 m2, 1 cajón de est...",Apartamento,"Pueblo de los Reyes, Coyoacán, Mexico D.F.",Coyoacán,Distrito Federal,0.0,2.0,1.0,...,100.0,23620.0,19.332829,-99.152913,2015-05-30 00:00:00,0.0,0.0,0.0,0.0,1.0
3,299321,departamento en venta en acapulco,<p> raíces dv001 precioso departamento tipo k...,Apartamento,NaN,Acapulco de Juárez,Guerrero,2.0,2.0,2.0,...,86.0,129347.0,16.860487,-99.878383,2015-04-02 00:00:00,0.0,0.0,0.0,0.0,0.0
4,173570,bonita casa sola equipada de dos niveles en lo...,"<p>casa sola, bonita de dos rec&aacute;maras u...",Casa,CEDROS,Tultitlán,Edo. de México,10.0,2.0,1.0,...,76.0,57125.0,19.640482,-99.127273,2013-08-15 00:00:00,0.0,0.0,0.0,1.0,1.0


In [9]:
len(test)

60000

# Preparacion del set de datos de entrenamiento (features)

La idea es preparar el set con los datos para exportar y que despues el modelo simplemente levante, separe en set de entrenamiento y test, entrene y devuelva una prediccion.

Se procede a calcular features. Cada feature se agregara al DataFrame final que tendra que levantar despues el modelo. Ojo que dentro de este DataFrame tambien va a estar el precio (que es el label).

In [10]:
train_set = pd.DataFrame()

In [11]:
test_set = pd.DataFrame()

Primero de todo, se agrega el id y el precio asocidado. A partir de esto, se agregaran los features que se consideren necesarios.

In [12]:
train_set["id"] = data["id"]
train_set["precio"] = data["precio"]

In [13]:
train_set.head()

,id,precio
0,254099,2273000.0
1,53461,3600000.0
2,247984,1200000.0
3,209067,650000.0
4,185997,1150000.0


A partir de esto se generan features.

Igual pero para test_set. Solo que sin el precio.

In [14]:
test_set["id"] = test["id"]

In [15]:
test_set.head()

,id
0,4941
1,51775
2,115253
3,299321
4,173570


In [16]:
len(test_set)

60000

# Se codifican las variables categoricas

In [17]:
data['train'] = True
test['train'] = False
combined = pd.concat([data, test], sort = True)

In [18]:
train_set['train'] = True
test_set["train"] = False
combined_set = pd.concat([train_set, test_set], sort = False)

In [19]:
len(combined_set)

300000

In [20]:
combined['tipodepropiedad'].nunique()

24

In [21]:
combined['ciudad'].nunique()

921

Ciudad tiene demasiados valores posibles para OneHotEncoding

In [22]:
combined['provincia'].nunique()

32

In [23]:
combined_set['tipodepropiedad'] = combined['tipodepropiedad']
combined_set['provincia'] = combined['provincia']

In [24]:
len(combined_set)

300000

In [25]:
var_categoricas = ['tipodepropiedad', 'provincia']

In [26]:
one_hot_enc = ce.OneHotEncoder(handle_unknown = 'ignore')
one_hot_encoded = one_hot_enc.fit_transform(combined_set[var_categoricas])
one_hot_encoded.columns

Index(['tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_23',
  

In [27]:
combined_set = combined_set.join(one_hot_encoded)
combined_set.columns

Index(['id', 'precio', 'train', 'tipodepropiedad', 'provincia',
       'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'p

In [28]:
len(combined_set)

420000

In [29]:
len(train_set)

240000

In [30]:
len(test_set)

60000

In [31]:
train_set = combined_set[combined_set["train"] == True]
test_set = combined_set[combined_set["train"] == False]

In [32]:
len(train_set)

300000

In [33]:
len(test_set)

120000

In [34]:
train_set = train_set.drop(columns = ['tipodepropiedad', 'provincia' , 'train'])

In [35]:
train_set.columns

Index(['id', 'precio', 'tipodepropiedad_1', 'tipodepropiedad_2',
       'tipodepropiedad_3', 'tipodepropiedad_4', 'tipodepropiedad_5',
       'tipodepropiedad_6', 'tipodepropiedad_7', 'tipodepropiedad_8',
       'tipodepropiedad_9', 'tipodepropiedad_10', 'tipodepropiedad_11',
       'tipodepropiedad_12', 'tipodepropiedad_13', 'tipodepropiedad_14',
       'tipodepropiedad_15', 'tipodepropiedad_16', 'tipodepropiedad_17',
       'tipodepropiedad_18', 'tipodepropiedad_19', 'tipodepropiedad_20',
       'tipodepropiedad_21', 'tipodepropiedad_22', 'tipodepropiedad_23',
       'tipodepropiedad_24', 'tipodepropiedad_25', 'provincia_1',
       'provincia_2', 'provincia_3', 'provincia_4', 'provincia_5',
       'provincia_6', 'provincia_7', 'provincia_8', 'provincia_9',
       'provincia_10', 'provincia_11', 'provincia_12', 'provincia_13',
       'provincia_14', 'provincia_15', 'provincia_16', 'provincia_17',
       'provincia_18', 'provincia_19', 'provincia_20', 'provincia_21',
       'provincia_

In [36]:
test_set = test_set.drop(columns = ['tipodepropiedad', 'precio' , 'provincia', 'train'])

In [37]:
test_set.columns

Index(['id', 'tipodepropiedad_1', 'tipodepropiedad_2', 'tipodepropiedad_3',
       'tipodepropiedad_4', 'tipodepropiedad_5', 'tipodepropiedad_6',
       'tipodepropiedad_7', 'tipodepropiedad_8', 'tipodepropiedad_9',
       'tipodepropiedad_10', 'tipodepropiedad_11', 'tipodepropiedad_12',
       'tipodepropiedad_13', 'tipodepropiedad_14', 'tipodepropiedad_15',
       'tipodepropiedad_16', 'tipodepropiedad_17', 'tipodepropiedad_18',
       'tipodepropiedad_19', 'tipodepropiedad_20', 'tipodepropiedad_21',
       'tipodepropiedad_22', 'tipodepropiedad_23', 'tipodepropiedad_24',
       'tipodepropiedad_25', 'provincia_1', 'provincia_2', 'provincia_3',
       'provincia_4', 'provincia_5', 'provincia_6', 'provincia_7',
       'provincia_8', 'provincia_9', 'provincia_10', 'provincia_11',
       'provincia_12', 'provincia_13', 'provincia_14', 'provincia_15',
       'provincia_16', 'provincia_17', 'provincia_18', 'provincia_19',
       'provincia_20', 'provincia_21', 'provincia_22', 'provincia_2

In [38]:
len(test)

60000

In [39]:
len(test_set)

120000

In [40]:
test_set = test_set.drop_duplicates("id")

In [41]:
len(test_set)

60000

In [42]:
train_set = train_set.drop_duplicates("id")

In [43]:
len(train_set)

240000

## Se agregan todas las columnas numericas que ya se tienen

In [44]:
data.dtypes

id                               int32
titulo                          object
descripcion                     object
tipodepropiedad               category
direccion                       object
ciudad                          object
provincia                     category
antiguedad                     float64
habitaciones                   float64
garages                        float64
banos                          float64
metroscubiertos                float64
metrostotales                  float64
idzona                          object
lat                            float64
lng                            float64
fecha                           object
gimnasio                       float64
usosmultiples                  float64
piscina                        float64
escuelascercanas               float64
centroscomercialescercanos     float64
precio                         float64
train                             bool
dtype: object

In [45]:
data_num = data.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [46]:
train_set = train_set.merge(data_num, on = ["id"], how = "inner")
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,254099,2273000.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,53461,3600000.0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
2,247984,1200000.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,209067,650000.0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0
4,185997,1150000.0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0


In [47]:
test_num = test.loc[:,["id", "usosmultiples", "piscina", "escuelascercanas",
                      "centroscomercialescercanos"]]

In [48]:
test_set = test_set.merge(test_num, on = ["id"], how = "inner")
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,provincia_27,provincia_28,provincia_29,provincia_30,provincia_31,provincia_32,usosmultiples,piscina,escuelascercanas,centroscomercialescercanos
0,4941,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,51775,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
2,115253,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,1.0
3,299321,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,0.0,0.0
4,173570,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0.0,0.0,1.0,1.0


## Se agregan el resto de las variables

In [49]:
data_num_fill = data.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [50]:
train_set = train_set.merge(data_num_fill, on = 'id', how = 'inner')
train_set.head()

,id,precio,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,254099,2273000.0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,NaN,80.0,80.0,1.0,2.0,NaN,NaN
1,53461,3600000.0,0,1,0,0,0,0,0,0,...,0.0,1.0,1.0,10.0,180.0,268.0,2.0,2.0,19.310205,-99.227655
2,247984,1200000.0,0,0,1,0,0,0,0,0,...,0.0,0.0,0.0,5.0,166.0,144.0,2.0,2.0,NaN,NaN
3,209067,650000.0,0,0,1,0,0,0,0,0,...,0.0,1.0,1.0,1.0,67.0,63.0,1.0,1.0,19.301890,-99.688015
4,185997,1150000.0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,10.0,95.0,95.0,1.0,1.0,NaN,NaN


In [51]:
test_num_fill = test.loc[:, ["id", "antiguedad", "metrostotales", "metroscubiertos", 'garages', 'banos', 'lat', 'lng']]

In [52]:
test_set = test_set.merge(test_num_fill, on = 'id', how = 'inner')
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,piscina,escuelascercanas,centroscomercialescercanos,antiguedad,metrostotales,metroscubiertos,garages,banos,lat,lng
0,4941,1,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,29.0,NaN,300.0,NaN,4.0,19.408668,-99.246767
1,51775,0,1,0,0,0,0,0,0,0,...,0.0,0.0,0.0,NaN,67.0,67.0,1.0,1.0,21.032480,-89.592424
2,115253,0,0,1,0,0,0,0,0,0,...,0.0,0.0,1.0,0.0,100.0,87.0,1.0,2.0,19.332829,-99.152913
3,299321,0,0,1,0,0,0,0,0,0,...,0.0,0.0,0.0,2.0,86.0,86.0,2.0,2.0,16.860487,-99.878383
4,173570,1,0,0,0,0,0,0,0,0,...,0.0,1.0,1.0,10.0,76.0,80.0,1.0,1.0,19.640482,-99.127273


In [53]:
len(test_set)

60000

In [54]:
train_set.isnull().sum().sum()

423384

# Nuevos features

# Promedio de metroscubiertos por tipo de propiedad

In [55]:
metros_mean = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [56]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,117.048657
1,Bodega comercial,251.153639
2,Casa,194.026911
3,Casa en condominio,187.697231
4,Casa uso de suelo,254.068966


In [57]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,117.048657
1,Bodega comercial,251.153639
2,Casa,194.026911
3,Casa en condominio,187.697231
4,Casa uso de suelo,254.068966


In [58]:
data = data.merge(metros_mean, on = 'tipodepropiedad')

In [59]:
train_set = train_set.merge(data[['id', 'metroscubiertos_mean_tipodeprop']], on = "id")

### Lo mismo pero para test

In [60]:
metros_mean = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "mean"}).reset_index()

In [61]:
metros_mean.head()

,tipodepropiedad,metroscubiertos
0,Apartamento,116.138123
1,Bodega comercial,250.683849
2,Casa,193.814888
3,Casa en condominio,186.547166
4,Casa uso de suelo,241.857895


In [62]:
metros_mean.rename(columns = {"metroscubiertos" : "metroscubiertos_mean_tipodeprop"}, inplace = True)
metros_mean.head()

,tipodepropiedad,metroscubiertos_mean_tipodeprop
0,Apartamento,116.138123
1,Bodega comercial,250.683849
2,Casa,193.814888
3,Casa en condominio,186.547166
4,Casa uso de suelo,241.857895


In [63]:
test = test.merge(metros_mean, on = 'tipodepropiedad')

In [64]:
test_set = test_set.merge(test[['id', 'metroscubiertos_mean_tipodeprop']], on = "id", how = "left")

In [65]:
len(test_set)

60000

# Varianza de metroscubiertos por tipo de propiedad

In [66]:
metros_var = data.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [67]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)

In [68]:
data = data.merge(metros_var, on = 'tipodepropiedad')

In [69]:
train_set = train_set.merge(data[['id', 'metroscubiertos_var_tipodeprop']], on = "id")

### Lo mismo pero para test

In [70]:
metros_var = test.groupby(by = 'tipodepropiedad').agg({"metroscubiertos" : "var"}).reset_index()

In [71]:
metros_var.rename(columns = {"metroscubiertos" : "metroscubiertos_var_tipodeprop"}, inplace = True)

In [72]:
test = test.merge(metros_var, on = 'tipodepropiedad')

In [73]:
test_set = test_set.merge(test[['id', 'metroscubiertos_var_tipodeprop']], on = "id", how = "left")

In [74]:
len(test_set)

60000

# Precio promedio según tipo de propiedad

In [75]:
precio_mean = data.groupby(by = 'tipodepropiedad').agg({"precio" : "mean"}).reset_index()

In [76]:
precio_mean.head()

,tipodepropiedad,precio
0,Apartamento,2.763769e+06
1,Bodega comercial,2.693758e+06
2,Casa,2.398158e+06
3,Casa en condominio,2.898927e+06
4,Casa uso de suelo,3.732469e+06


In [77]:
precio_mean.rename(columns = {"precio" : "precio_mean_tipodeprop"}, inplace = True)
precio_mean.head()

,tipodepropiedad,precio_mean_tipodeprop
0,Apartamento,2.763769e+06
1,Bodega comercial,2.693758e+06
2,Casa,2.398158e+06
3,Casa en condominio,2.898927e+06
4,Casa uso de suelo,3.732469e+06


In [78]:
data = data.merge(precio_mean, on = 'tipodepropiedad')

In [79]:
train_set = train_set.merge(data[['id', 'precio_mean_tipodeprop']], on = "id")

### Lo mismo pero para test

In [80]:
test = test.merge(precio_mean, on = 'tipodepropiedad')

In [81]:
test_set = test_set.merge(test[['id', 'precio_mean_tipodeprop']], on = "id", how = "left")

In [82]:
len(test_set)

60000

# Varianza del precio segun tipo de propiedad

In [83]:
precio_var = data.groupby(by = 'tipodepropiedad').agg({"precio" : "var"}).reset_index()

In [84]:
precio_var.head()

,tipodepropiedad,precio
0,Apartamento,5.488593e+12
1,Bodega comercial,4.009818e+12
2,Casa,3.997227e+12
3,Casa en condominio,5.359409e+12
4,Casa uso de suelo,6.400712e+12


In [85]:
precio_var.rename(columns = {"precio" : "precio_var_tipodeprop"}, inplace = True)
precio_var.head()

,tipodepropiedad,precio_var_tipodeprop
0,Apartamento,5.488593e+12
1,Bodega comercial,4.009818e+12
2,Casa,3.997227e+12
3,Casa en condominio,5.359409e+12
4,Casa uso de suelo,6.400712e+12


In [86]:
data = data.merge(precio_var, on = 'tipodepropiedad')

In [87]:
train_set = train_set.merge(data[['id', 'precio_var_tipodeprop']], on = "id")

### Lo mismo pero para test

In [88]:
test = test.merge(precio_var, on = 'tipodepropiedad')

In [89]:
test_set = test_set.merge(test[['id', 'precio_var_tipodeprop']], on = "id", how = "left")

In [90]:
len(test_set)

60000

# Precio promedio  para intervalos de metros

Da muy mal este feature

In [91]:
# bins = 10
# precio_mean_mts = data.groupby(by = pd.cut(data['metroscubiertos'], bins)).agg({"precio" : "mean"}).reset_index()
# data['metroscubiertos_bin'] = pd.cut(data["metroscubiertos"], bins)
# precio_mean_mts.head()

In [92]:
# precio_mean_mts.rename(columns = {"metroscubiertos":"metroscubiertos_bin", "precio":"precio_mean_mts"}, inplace = True)
# precio_mean_mts.head()

In [93]:
# data = data.merge(precio_mean_mts, on = 'metroscubiertos_bin')

In [94]:
# train_set = train_set.merge(data[['id', 'precio_mean_mts']], on = "id")

In [95]:
# precio_mean_mts['metroscubiertos_bin']

In [96]:
# bins = pd.IntervalIndex.from_tuples([(14.576, 57.4),
#       (57.4, 99.8),
#      (99.8, 142.2),
#     (142.2, 184.6),
#     (184.6, 227.0),
#     (227.0, 269.4),
#     (269.4, 311.8),
#     (311.8, 354.2),
#     (354.2, 396.6),
#     (396.6, 439.0)], closed = 'right' )
# test_set['metroscubiertos_bin'] = pd.cut(test_set['metroscubiertos'], bins)

In [97]:
# test_set['metroscubiertos_bin'].head()

In [98]:
# precio_mean_mts.head()

In [99]:
# test_set = test_set.merge(precio_mean_mts, on = "metroscubiertos_bin", how = 'left')

In [100]:
# test_set.head()

# Varianza de precio para intervalos de metros

Da muy mal este feature

In [101]:
# bins = 10
# precio_var_mts = data.groupby(by = pd.cut(data['metroscubiertos'], bins)).agg({"precio" : "var"}).reset_index()
# data['metroscubiertos_bin'] = pd.cut(data["metroscubiertos"], bins)
# precio_var_mts.head()

In [102]:
# precio_var_mts.rename(columns = {"metroscubiertos":"metroscubiertos_bin", "precio":"precio_var_mts"}, inplace = True)
# precio_var_mts.head()

In [103]:
# data = data.merge(precio_var_mts, on = 'metroscubiertos_bin')

In [104]:
# train_set = train_set.merge(data[['id', 'precio_var_mts']], on = "id")

In [105]:
# precio_var_mts['metroscubiertos_bin']

In [106]:
# bins = pd.IntervalIndex.from_tuples([(14.576, 57.4),
#       (57.4, 99.8),
#      (99.8, 142.2),
#     (142.2, 184.6),
#     (184.6, 227.0),
#     (227.0, 269.4),
#     (269.4, 311.8),
#     (311.8, 354.2),
#     (354.2, 396.6),
#     (396.6, 439.0)], closed = 'right' )
# test_set['metroscubiertos_bin'] = pd.cut(test_set['metroscubiertos'], bins)

In [107]:
# test_set['metroscubiertos_bin'].head()

In [108]:
# precio_var_mts.head()

In [109]:
# test_set = test_set.merge(precio_var_mts, on = "metroscubiertos_bin", how = 'left')

In [110]:
# test_set.head()

In [111]:
# del test_set['metroscubiertos_bin']

# Precio promedio por latitud y longitud

In [112]:
# bins = 10
# data['lat_bin'] = pd.cut(data["lat"], bins)
# data['lng_bin'] = pd.cut(data['lng'], bins)
# precio_mean_lat_lng = data.groupby(by = ['lat_bin', 'lng_bin']).agg({"precio" : "mean"}).reset_index()
# precio_mean_lat_lng.head()

In [113]:
# precio_mean_lat_lng.rename(columns = {"precio":"precio_mean_lat_lng_bin"}, inplace = True)
# precio_mean_lat_lng.head()S

In [114]:
# data = data.merge(precio_mean_lat_lng, on = ['lat_bin', 'lng_bin'])
# data['precio_mean_lat_lng_bin']

In [115]:
# train_set = train_set.merge(data[['id', 'precio_mean_lat_lng_bin']], on = "id")

In [116]:
# precio_mean_lat_lng['lat_bin'].unique()

In [117]:
# precio_mean_lat_lng['lng_bin'].unique()

In [118]:
# bins = pd.IntervalIndex.from_tuples([(-80.79, -64.261), (-64.261, -47.896), 
# (-47.896, -31.531), (-31.531, -15.165), 
# (-15.165, 1.2), (1.2, 17.565), 
# (17.565, 33.93), (33.93, 50.296), 
# (50.296, 66.661), (66.661, 83.026)], closed = 'right' )
# test_set['lat_bin'] = pd.cut(test_set['lat'], bins)

In [119]:
# bins = pd.IntervalIndex.from_tuples([(-126.106, -101.17), (-101.17, -76.48), 
# (-76.48, -51.791), (-51.791, -27.101), 
# (-27.101, -2.412), (-2.412, 22.278), 
# (22.278, 46.967), (46.967, 71.657), 
# (71.657, 96.346), (96.346, 121.036)], closed = 'right' )
# test_set['lng_bin'] = pd.cut(test_set['lng'], bins)

In [120]:
# test_set['lng_bin']

In [121]:
# test_set['lat_bin']

In [122]:
# precio_mean_lat_lng['lat_bin']

In [123]:
# test_set = test_set.merge(precio_mean_lat_lng, on = ["lat_bin"], how = 'left')

Por que lat_bin no anda pero lng_bin si? Si son exactamente el mismo tipo

In [124]:
# test_set.head()

In [125]:
# del test['lat_bin']
# del test['lng_bin']

### Feature total de antiguedad por anio

In [126]:
data.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239944,239945,239946,239947,239948,239949,239950,239951,239952,239953
id,254099,185997,126147,44962,103293,73348,257302,127666,212847,224685,...,204578,10447,189648,257049,70279,169168,104078,136796,16605,245563
titulo,depto. tipo a-402,paseos del sol,departamento en venta taxqueña,pre- venta preciosos depas 2 recamaras con sub...,"departamento en venta, san pedro garza garcia,...","oportunidad, departamento col del valle, 3 rec...",NaN,hermoso departamento!!!,departamento en arcos de zapopan,renta en acapulco con alberca prvada vista a b...,...,granja en venta granjas montenegro,00477rmvsu campo agricola san miguel de horcas...,"hermosa granja de 10,000 m2 en el sauz, a 40 m...","terreno / lote en venta, alvarado, veracruz",terreno/lote - cerrada dr. duran,terreno/lote - paseo ex hacienda barbosa,terreno/lote - privada de rio nasas,terreno/lote - 3er. cerrada de juan escutia,hospedaje en venta en 23 de julio,NaN
descripcion,"depto. interior de 80.15m2, consta de sala com...",bonito departamento en excelentes condiciones ...,"amplio departamento, estancia de sala y comedo...",<p>pre-venta de preciosos departamento ecologi...,"departamento nuevo ,256 m2 de construccion,un ...",magnifico departamento con excelente distribuc...,-- hav1407e-285 -- excelente departamento e...,hermoso departamento con acabados de muy buen ...,bonito departamento muy bien ubicado en planta...,bonito departamento cerca de sam´s club farall...,...,"bonita granja con gran ubicacion,ideal para de...","<p>excelente campo agricola 20 has, rumbo a sa...","10,000 m² de terreno, 60 m² de construcción, v...",chatea con nosotros para conocer mas sobre est...,"excelente predio para demoler, ideal para cons...","oportunidad de terreno, plano, con servicio de...",excelente opción de terreno para las familias ...,bonito terreno ubicado en una zona de crecimie...,venta edificio en ciudad del carmen. excelente...,"4 lugares de estacionamiento (40 m2 en total, ..."
tipodepropiedad,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,...,Huerta,Huerta,Huerta,Lote,Lote,Lote,Lote,Lote,Hospedaje,Garage
direccion,Avenida Division del Norte 2005,PASEOS DEL SOL,Condominio Tlalpan 2B,BUENAVISTA DEPTOS CON SUBSIDIO,NaN,Pazaje Santa Cruz,Rio Mante .,LAS COLONIAS,ARCO PERTINAX,RANCHO ACAPULCO,...,Camino a la Presa # 5600,CARRETERA A SAN MIGUEL DE HORCASITAS KM 19,El Sauz,NaN,NaN,NaN,NaN,NaN,"Machiche, 23 de Julio, Ciudad del Carmen",Mineria
ciudad,Benito Juárez,Zapopan,Coyoacán,Villa de Alvarez,San Pedro Garza García,Benito Juárez,Ciudad Madero,Atizapán de Zaragoza,Zapopan,Acapulco de Juárez,...,El Salto,San Miguel de Horcasitas,Chihuahua,Alvarado,Cuauhtémoc,Zinacantepec,Metepec,Toluca,Carmen,Miguel Hidalgo
provincia,Distrito Federal,Jalisco,Distrito Federal,Colima,Nuevo León,Distrito Federal,Tamaulipas,Edo. de México,Jalisco,Guerrero,...,Jalisco,Sonora,Chihuahua,Veracruz,Distrito Federal,Edo. de México,Edo. de México,Edo. de México,Campeche,Distrito Federal
antiguedad,NaN,10,5,1,NaN,5,0,20,10,3,...,10,3,10,0,NaN,NaN,NaN,NaN,NaN,NaN
habitaciones,2,2,2,2,3,3,2,2,2,2,...,4,NaN,2,NaN,NaN,NaN,NaN,NaN,7,NaN
garages,1,1,1,1,2,2,1,1,1,1,...,0,0,0,0,NaN,NaN,NaN,NaN,0,0


In [127]:
ant = data
ant['count'] = 1
ant = ant.groupby(by='antiguedad').sum()[['count','habitaciones','id']]
#ant.sort_values(ascending=False).tail(5)
ant = ant.reset_index()
ant.head()

,antiguedad,count,habitaciones,id
0,0.0,50330,126555.0,7.520635e+09
1,1.0,12353,32263.0,1.857008e+09
2,2.0,5059,13537.0,7.697436e+08
3,3.0,5616,15068.0,8.434886e+08
4,4.0,7944,21356.0,1.196237e+09


In [128]:
def hab_ant(row):
    #if(row['antiguedad'] <= 10 ):
        return row['count'] * 1
    #return row['habitaciones'] * 0.5

In [129]:
ant2 = ant
ant2['c_hab_ant'] = ant2.apply(hab_ant,axis=1)
ant2.head(15)

,antiguedad,count,habitaciones,id,c_hab_ant
0,0.0,50330,126555.0,7.520635e+09,50330.0
1,1.0,12353,32263.0,1.857008e+09,12353.0
2,2.0,5059,13537.0,7.697436e+08,5059.0
3,3.0,5616,15068.0,8.434886e+08,5616.0
4,4.0,7944,21356.0,1.196237e+09,7944.0
5,5.0,33265,89649.0,4.970438e+09,33265.0
6,6.0,3169,8520.0,4.761291e+08,3169.0
7,7.0,2046,5513.0,3.130939e+08,2046.0
8,8.0,2418,6760.0,3.712067e+08,2418.0
9,9.0,1065,2905.0,1.624691e+08,1065.0


In [130]:
data = data.drop(columns='count')

In [131]:
ant3 = ant2[['c_hab_ant','antiguedad']]
ant3.head()

train_set = train_set.merge(ant3,on = 'antiguedad',how = 'left')
train_set.transpose()

test_set = test_set.merge(ant3,on = 'antiguedad',how = 'left')
test_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,59990,59991,59992,59993,59994,59995,59996,59997,59998,59999
id,4.941000e+03,5.177500e+04,1.152530e+05,2.993210e+05,1.735700e+05,3.086200e+04,2.444710e+05,1.277940e+05,7.155800e+04,2.180110e+05,...,2.056250e+05,2.842660e+05,7.024400e+04,5.977600e+04,7.910000e+04,7.509400e+04,1.718470e+05,1.383130e+05,2.712680e+05,7.261200e+04
tipodepropiedad_1,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
tipodepropiedad_2,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_3,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
tipodepropiedad_7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_8,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


### Feature: antiguedad por provincia

In [132]:
data.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239944,239945,239946,239947,239948,239949,239950,239951,239952,239953
id,254099,185997,126147,44962,103293,73348,257302,127666,212847,224685,...,204578,10447,189648,257049,70279,169168,104078,136796,16605,245563
titulo,depto. tipo a-402,paseos del sol,departamento en venta taxqueña,pre- venta preciosos depas 2 recamaras con sub...,"departamento en venta, san pedro garza garcia,...","oportunidad, departamento col del valle, 3 rec...",NaN,hermoso departamento!!!,departamento en arcos de zapopan,renta en acapulco con alberca prvada vista a b...,...,granja en venta granjas montenegro,00477rmvsu campo agricola san miguel de horcas...,"hermosa granja de 10,000 m2 en el sauz, a 40 m...","terreno / lote en venta, alvarado, veracruz",terreno/lote - cerrada dr. duran,terreno/lote - paseo ex hacienda barbosa,terreno/lote - privada de rio nasas,terreno/lote - 3er. cerrada de juan escutia,hospedaje en venta en 23 de julio,NaN
descripcion,"depto. interior de 80.15m2, consta de sala com...",bonito departamento en excelentes condiciones ...,"amplio departamento, estancia de sala y comedo...",<p>pre-venta de preciosos departamento ecologi...,"departamento nuevo ,256 m2 de construccion,un ...",magnifico departamento con excelente distribuc...,-- hav1407e-285 -- excelente departamento e...,hermoso departamento con acabados de muy buen ...,bonito departamento muy bien ubicado en planta...,bonito departamento cerca de sam´s club farall...,...,"bonita granja con gran ubicacion,ideal para de...","<p>excelente campo agricola 20 has, rumbo a sa...","10,000 m² de terreno, 60 m² de construcción, v...",chatea con nosotros para conocer mas sobre est...,"excelente predio para demoler, ideal para cons...","oportunidad de terreno, plano, con servicio de...",excelente opción de terreno para las familias ...,bonito terreno ubicado en una zona de crecimie...,venta edificio en ciudad del carmen. excelente...,"4 lugares de estacionamiento (40 m2 en total, ..."
tipodepropiedad,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,...,Huerta,Huerta,Huerta,Lote,Lote,Lote,Lote,Lote,Hospedaje,Garage
direccion,Avenida Division del Norte 2005,PASEOS DEL SOL,Condominio Tlalpan 2B,BUENAVISTA DEPTOS CON SUBSIDIO,NaN,Pazaje Santa Cruz,Rio Mante .,LAS COLONIAS,ARCO PERTINAX,RANCHO ACAPULCO,...,Camino a la Presa # 5600,CARRETERA A SAN MIGUEL DE HORCASITAS KM 19,El Sauz,NaN,NaN,NaN,NaN,NaN,"Machiche, 23 de Julio, Ciudad del Carmen",Mineria
ciudad,Benito Juárez,Zapopan,Coyoacán,Villa de Alvarez,San Pedro Garza García,Benito Juárez,Ciudad Madero,Atizapán de Zaragoza,Zapopan,Acapulco de Juárez,...,El Salto,San Miguel de Horcasitas,Chihuahua,Alvarado,Cuauhtémoc,Zinacantepec,Metepec,Toluca,Carmen,Miguel Hidalgo
provincia,Distrito Federal,Jalisco,Distrito Federal,Colima,Nuevo León,Distrito Federal,Tamaulipas,Edo. de México,Jalisco,Guerrero,...,Jalisco,Sonora,Chihuahua,Veracruz,Distrito Federal,Edo. de México,Edo. de México,Edo. de México,Campeche,Distrito Federal
antiguedad,NaN,10,5,1,NaN,5,0,20,10,3,...,10,3,10,0,NaN,NaN,NaN,NaN,NaN,NaN
habitaciones,2,2,2,2,3,3,2,2,2,2,...,4,NaN,2,NaN,NaN,NaN,NaN,NaN,7,NaN
garages,1,1,1,1,2,2,1,1,1,1,...,0,0,0,0,NaN,NaN,NaN,NaN,0,0


In [133]:
#Mergea el la columna indicada en 'on', tomando el df pasado como parámetro(df_to_use)
#how_feature indica que tipo de join se hace.
#El merge lo hace sobre 'train_set' y 'test_set'
def merge_df(df_to_use, on_feature,how_feature):
    print("dataframe a usar:"+str(df_to_use))
    print("feature a usar:"+on_feature)
   # print("[test_set] Cantidad init: "+str(len(test_set.columns)))
    #print("[train_set] Cantidad init: "+str(len(train_set.columns)))
    test_set.merge(df_to_use, on = on_feature, how = how_feature)
    train_set.merge(df_to_use, on = on_feature, how = how_feature)
    #print("[test_set] Cantidad init: "+str(len(test_set.columns)))
    #print("[train_set] Cantidad init: "+str(len(train_set.columns)))

In [134]:
df = data.groupby(by = 'provincia').agg({'antiguedad':'mean'}).reset_index()
df.sort_values(['antiguedad'], ascending = False, inplace = True)
df=df.rename(columns={"antiguedad":"antiguedad_prov"})
df.head()

,provincia,antiguedad_prov
8,Distrito Federal,12.189608
10,Edo. de México,10.027827
12,Guerrero,7.605657
18,Nuevo León,7.539231
0,Aguascalientes,7.524580


In [135]:
data = data.merge(df,on='provincia',how='inner')
data.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239791,239792,239793,239794,239795,239796,239797,239798,239799,239800
id,254099,126147,73348,116928,244353,198249,232207,237577,215118,171944,...,61898,208698,27706,190742,58226,10722,271397,2317,226592,57022
titulo,depto. tipo a-402,departamento en venta taxqueña,"oportunidad, departamento col del valle, 3 rec...",excelentes departamentos con exclente ubicación,NaN,departamento calle 6 agricola pantitlan q,NaN,departamento en venta en benito juárez,bonito depa en san miguel chapultepec. precio ...,nuevo al poniente y en parcialidades.,...,"terreno en colonia las palmas, zacatecas",terreno / lote en venta en guadalupe,terreno / lote en venta en jerez,terreno en venta en zacatecas,terreno el talancón,se vende local,"edificio en residencial boulevares , zacatecas","edificio en venta, zona centro, zacatecas, zac","edificio en residencial boulevares, zac. (ven...","local y departamento en venta en sombrerete, zac."
descripcion,"depto. interior de 80.15m2, consta de sala com...","amplio departamento, estancia de sala y comedo...",magnifico departamento con excelente distribuc...,"1 pb, 2 primer piso, 2 segundo piso. cada depa...","departamento nuevo, 98m2 ,interior, 2 recamara...","proyecto de departamentos de hasta 53 m2, en l...",-- rcv53n-295 --,"excelente ubicación, amplios espacios, muy ilu...","3 recamaras, 1 1/2 baños , sala, comedor, coci...",un nuevo estilo de vida al sur de la ciudad. \...,...,<p>boulevard heroes de chapultepec</p>,"terreno de 250m2 con excelente ubicacion, zona...","terreno a casa residencial, excelentes condic...","<p>terreno en venta en zacatecas, ubicado en ...",/ha. ha.<br /><br />,NaN,"<p>consta de 6 niveles, 15 cub&iacute;culos, e...",<table cellspacing=0 cellpadding=0 align=left>...,<table cellspacing=0 cellpadding=0 align=left>...,<table cellspacing=0 cellpadding=0 align=left>...
tipodepropiedad,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,Apartamento,...,Terreno,Terreno,Terreno,Terreno,Terreno comercial,Local Comercial,Edificio,Edificio,Edificio,Edificio
direccion,Avenida Division del Norte 2005,Condominio Tlalpan 2B,Pazaje Santa Cruz,Jesús Carranza No. 55,ILLINOIS 31 602,CALLE 6 NUMERO 106,PRIMERA CERRADA DE ROJAS 13,JUAN SANCEHZ AZCONA,"GELATI 42, DEPTO. 1",Av. de las Torres,...,BOULEVARD HEROES DE CHAPULTEPEC,Grieta,Los Arquitos,NaN,NaN,AV GARCIA SALINAS,"CALLE UNO, ESQ. JULIO RUELAS","CALLE JUAN DE TOLOSA #1122, ZONA CENTRO, ZACAT...","CALLE UNO, BOULEVARES, ZACATECAS, ZAC.","JARDIN HIDALGO #204, ZONA CENTRO, SOMBRERETE, ..."
ciudad,Benito Juárez,Coyoacán,Benito Juárez,Cuauhtémoc,Benito Juárez,Iztacalco,Cuauhtémoc,Benito Juárez,Miguel Hidalgo,Alvaro Obregón,...,Zacatecas,Guadalupe,Jerez,Jerez,Morelos,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Sombrerete
provincia,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,Distrito Federal,...,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas,Zacatecas
antiguedad,NaN,5,5,20,0,0,4,6,10,0,...,0,NaN,NaN,6,0,11,0,0,10,10
habitaciones,2,2,3,2,2,2,2,3,3,2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
garages,1,1,2,0,2,1,1,2,1,2,...,0,0,0,NaN,0,0,0,0,0,0


In [136]:
train_set = train_set.merge(data[['id','antiguedad_prov']], on = 'id', how = 'left')
test_set = test_set.merge(data[['id','antiguedad_prov']], on = 'id', how = 'left')

In [137]:
train_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,239944,239945,239946,239947,239948,239949,239950,239951,239952,239953
id,2.540990e+05,5.346100e+04,2.479840e+05,2.090670e+05,1.859970e+05,1.261470e+05,1.392330e+05,5.013000e+03,4.496200e+04,1.345370e+05,...,8.749800e+04,1.373370e+05,5.488600e+04,2.078920e+05,1.102680e+05,1.198790e+05,2.591780e+05,1.319320e+05,1.468670e+05,1.219580e+05
precio,2.273000e+06,3.600000e+06,1.200000e+06,6.500000e+05,1.150000e+06,1.100000e+06,1.150000e+06,4.200000e+06,3.100000e+05,6.200000e+06,...,1.495000e+06,1.450000e+06,5.500000e+06,1.650000e+06,1.350000e+06,6.500000e+05,1.940000e+06,3.400000e+06,2.890000e+06,3.650000e+06
tipodepropiedad_1,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00
tipodepropiedad_2,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_3,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
tipodepropiedad_4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_8,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


In [138]:
test_set.transpose()

,0,1,2,3,4,5,6,7,8,9,...,59990,59991,59992,59993,59994,59995,59996,59997,59998,59999
id,4.941000e+03,5.177500e+04,1.152530e+05,2.993210e+05,1.735700e+05,3.086200e+04,2.444710e+05,1.277940e+05,7.155800e+04,2.180110e+05,...,2.056250e+05,2.842660e+05,7.024400e+04,5.977600e+04,7.910000e+04,7.509400e+04,1.718470e+05,1.383130e+05,2.712680e+05,7.261200e+04
tipodepropiedad_1,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
tipodepropiedad_2,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_3,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,...,1.000000e+00,1.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_5,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_6,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00
tipodepropiedad_7,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_8,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
tipodepropiedad_9,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00


# Se exporta para entrenar

## NO hace falta eliminar NULLs

In [139]:
len(train_set)

239954

In [140]:
train_set.to_csv("train_set_xgb.csv", index = False)

In [141]:
test_set.head()

,id,tipodepropiedad_1,tipodepropiedad_2,tipodepropiedad_3,tipodepropiedad_4,tipodepropiedad_5,tipodepropiedad_6,tipodepropiedad_7,tipodepropiedad_8,tipodepropiedad_9,...,garages,banos,lat,lng,metroscubiertos_mean_tipodeprop,metroscubiertos_var_tipodeprop,precio_mean_tipodeprop,precio_var_tipodeprop,c_hab_ant,antiguedad_prov
0,4941,1,0,0,0,0,0,0,0,0,...,NaN,4.0,19.408668,-99.246767,193.814888,9797.989886,2.398158e+06,3.997227e+12,196.0,NaN
1,51775,0,1,0,0,0,0,0,0,0,...,1.0,1.0,21.032480,-89.592424,116.138123,4425.701226,2.763769e+06,5.488593e+12,NaN,NaN
2,115253,0,0,1,0,0,0,0,0,0,...,1.0,2.0,19.332829,-99.152913,116.138123,4425.701226,2.763769e+06,5.488593e+12,50330.0,NaN
3,299321,0,0,1,0,0,0,0,0,0,...,2.0,2.0,16.860487,-99.878383,116.138123,4425.701226,2.763769e+06,5.488593e+12,5059.0,NaN
4,173570,1,0,0,0,0,0,0,0,0,...,1.0,1.0,19.640482,-99.127273,193.814888,9797.989886,2.398158e+06,3.997227e+12,28844.0,NaN


In [142]:
len(test_set)

60000

In [143]:
test_set.to_csv("test_set_xgb.csv", index = False)